Cross Validation
===============

데이터세트 준비하기
-----------------

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Iris 데이터 세트 불러오기
iris = load_iris()

# Classification에 사용할 머신러닝 모델 불러오기
dt_clf = DecisionTreeClassifier()

# 전체 데이터로 학습
dt_clf.fit(iris.data, iris.target)

# 전체 데이터로 검증
pred = dt_clf.predict(iris.data)

# 검증 정확도 출력
# 학습한 데이터를 그대로 검증 데이터로 사용하면 정확도가 100%가 나오게 됩니다.
# 그러나 올바른 학습과 검증의 과정이 아닙니다.
print("검증 정확도(accuracy) : {0:.2f}%".format(accuracy_score(iris.target, pred) * 100))


# 전체 데이터를 학습 데이터와 검증 데이터로 나누어 머신러닝 모델을 평가합니다.
# 데이터 세트를 나눔으로써 머신러닝 모델을 좀 더 일반화할 수 있습니다.
X_train, X_test, Y_train, Y_test =  train_test_split(iris.data, iris.target, test_size = 0.25)

# 학습 데이터로 모델 학습 진행
dt_clf.fit(X_train, Y_train)  

# 검증 데이터로 성능 평가
pred = dt_clf.predict(X_test)

# 검증 정확도 출력
print("검증 정확도(accuracy) : {0:.2f}%".format( accuracy_score(Y_test, pred) * 100))   

검증 정확도(accuracy) : 100.00%
검증 정확도(accuracy) : 89.47%


K_fold 교차 검증
---------------

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Iris 데이터 세트 불러오기
iris = load_iris() 
# Classification에 사용할 모델 불러오기
dt_clf = DecisionTreeClassifier()

# 몇 개의 Fold로 나눌지 결정합니다.
n_iter = 0
kfold = KFold(n_splits = 5)
cv_accuracy = []

for train_idx, test_idx in kfold.split(iris.data):    # Data를 K만큼 나누기
    X_train, X_test = iris.data[train_idx], iris.data[test_idx] # 나눈 데이터 저장
    y_train, y_test = iris.target[train_idx], iris.target[test_idx]

    dt_clf.fit(X_train,y_train)     # 모델 학습
    pred = dt_clf.predict(X_test)   # 검증 데이터로 결과 예측
    n_iter += 1

    accuracy = np.round(accuracy_score(y_test, pred), 4)    # 각 Iter 별 정확도 측정
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    print("Iter : {0} Cross-Validation Accuracy : {1}, Train Data 크기 : {2}, Test Data 크기 : {3}"
          .format(n_iter, accuracy, train_size, test_size))

    cv_accuracy.append(accuracy)

print("평균 검증 정확도 : ", np.mean(cv_accuracy)) # 평균 검증 정확도 출력

Iter : 1 Cross-Validation Accuracy : 1.0, Train Data 크기 : 120, Test Data 크기 : 30
Iter : 2 Cross-Validation Accuracy : 0.9667, Train Data 크기 : 120, Test Data 크기 : 30
Iter : 3 Cross-Validation Accuracy : 0.9, Train Data 크기 : 120, Test Data 크기 : 30
Iter : 4 Cross-Validation Accuracy : 0.9333, Train Data 크기 : 120, Test Data 크기 : 30
Iter : 5 Cross-Validation Accuracy : 0.7333, Train Data 크기 : 120, Test Data 크기 : 30
평균 검증 정확도 :  0.9066599999999999


Stratified K-Fold 교차 검증
-----------------

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Iris 데이터 세트 불러오기
iris = load_iris()

# 데이터 분포를 확인하기 위해 DataFrame을 만들어줍니다.
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target

# Classification에 사용할 모델 불러오기
dt_clf = DecisionTreeClassifier()

# Iris 데이터는 3개의 Class로 이루어져 있습니다.
# 그러므로 3개의 Fold로 데이터를 나눕니다.
n_iter = 0
skf = StratifiedKFold(n_splits=3)
avg_acc = []

for train_idx, test_idx in skf.split(iris_df, iris_df['label']):    # iris 데이터에서 나누기
    # Iter 수 증가
    n_iter += 1
    
    # K 개수 만큼 Fold 나누기
    train_label = iris_df['label'].iloc[train_idx]                  
    test_label = iris_df['label'].iloc[test_idx]
    X_train, X_test = iris.data[train_idx], iris.data[test_idx]
    y_train, y_test = iris.target[train_idx], iris.target[test_idx]
    
    print("Iteration :", n_iter)
    print("--------------------")
    print("학습 데이터 분포 : \n", train_label.value_counts())
    print("--------------------")
    print("검증 데이터 분포 : \n", test_label.value_counts())
    print("--------------------")
    
    # 학습 데이터로모델 학습
    dt_clf.fit(X_train,y_train)
    
    # 검증 데이터로 성능 평가
    pred = dt_clf.predict(X_test)

    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    print("Iter : {0}, 정확도 : {1}%, 학습 데이터 개수 : {2}, 검증 데이터 개수 : {3}\n"
          .format(n_iter, accuracy*100, train_size, test_size))

    avg_acc.append(accuracy)
    
print("평균 검증 정확도 : {0:.4f}%".format(np.mean(avg_acc)* 100))

Iteration : 1
--------------------
학습 데이터 분포 : 
 2    33
1    33
0    33
Name: label, dtype: int64
--------------------
검증 데이터 분포 : 
 2    17
1    17
0    17
Name: label, dtype: int64
--------------------
Iter : 1, 정확도 : 98.04%, 학습 데이터 개수 : 99, 검증 데이터 개수 : 51

Iteration : 2
--------------------
학습 데이터 분포 : 
 2    33
1    33
0    33
Name: label, dtype: int64
--------------------
검증 데이터 분포 : 
 2    17
1    17
0    17
Name: label, dtype: int64
--------------------
Iter : 2, 정확도 : 92.16%, 학습 데이터 개수 : 99, 검증 데이터 개수 : 51

Iteration : 3
--------------------
학습 데이터 분포 : 
 2    34
1    34
0    34
Name: label, dtype: int64
--------------------
검증 데이터 분포 : 
 2    16
1    16
0    16
Name: label, dtype: int64
--------------------
Iter : 3, 정확도 : 97.92%, 학습 데이터 개수 : 102, 검증 데이터 개수 : 48

평균 검증 정확도 : 96.0400%


Data Preparation
---------------

In [7]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import KFold, train_test_split

# Load_digits()로 데이터를 불러오세요.
digits = load_digits()

# train_test_split()과 KFold()에 들어갈 파라미터를 설정해주세요.
# Train : Test 비율 =  7 : 3
# Fold의 개수 = 3
test_size = 0.3
n_splits = 3

# Kfold를 선언해주세요.
kfold = KFold(n_splits = n_splits)

# train_test_split()으로 데이터를 나눠주세요.
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size = test_size)

# K-Fold를 이용해 X_train을 3 Fold로 나눠보세요.
for train_idx, vali_idx in kfold.split(X_train):
    X_fold_train, X_fold_vali = digits.data[train_idx], digits.data[vali_idx]
    y_fold_train, y_fold_vali = digits.target[train_idx], digits.target[vali_idx]

Leave One Out(LOO)
-----------

In [1]:
# 전체 n개의 데이터셋 중 1개만을 validation에 사용함
import numpy as np
from sklearn.model_selection import LeaveOneOut
X = np.array([[1,2], [3,4], [5,6]])
loo = LeaveOneOut()
loo.get_n_splits(X)

for train_idx, test_idx in loo.split(X):
    print('TRAIN :',train_idx, 'TEST :', test_idx)
    X_train, X_test = X[train_idx], X[test_idx]
    print(X_train, X_test)

TRAIN : [1 2] TEST : [0]
[[3 4]
 [5 6]] [[1 2]]
TRAIN : [0 2] TEST : [1]
[[1 2]
 [5 6]] [[3 4]]
TRAIN : [0 1] TEST : [2]
[[1 2]
 [3 4]] [[5 6]]


Leave P Out(LPO)
---------

In [2]:
from sklearn.model_selection import LeavePOut
X = np.array([[1,2], [3,4], [5,6]])
lpo = LeavePOut(2)
lpo.get_n_splits(X)

for train_idx, test_idx in lpo.split(X):
    print('TRAIN :', train_idx, 'TEST :', test_idx)
    X_train, X_test = X[train_idx], X[test_idx]
    print(X_train, X_test)

TRAIN : [2] TEST : [0 1]
[[5 6]] [[1 2]
 [3 4]]
TRAIN : [1] TEST : [0 2]
[[3 4]] [[1 2]
 [5 6]]
TRAIN : [0] TEST : [1 2]
[[1 2]] [[3 4]
 [5 6]]


Shuffle-Split
-----------

In [3]:
from sklearn.model_selection import ShuffleSplit
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [3, 4], [5, 6]])
y = np.array([1, 2, 1, 2, 1, 2])
rs = ShuffleSplit(n_splits=5, test_size=.25, random_state=0)
rs.get_n_splits(X)
ShuffleSplit(n_splits=5, random_state=0, test_size=0.25, train_size=None)
for train_index, test_index in rs.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [1 3 0 4] TEST: [5 2]
TRAIN: [4 0 2 5] TEST: [1 3]
TRAIN: [1 2 4 0] TEST: [3 5]
TRAIN: [3 4 1 0] TEST: [5 2]
TRAIN: [3 5 1 0] TEST: [2 4]
